In [1]:
# export the model
# genres
# fix dataset and export it
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
import joblib
from sklearn.preprocessing import LabelEncoder

In [2]:
# genres_csv = pd.read_csv('data_by_genres_o.csv')
df = pd.read_csv('US.csv')

# create year column from release date 
df['year'] = pd.DatetimeIndex(df['release_date']).year

# drop categorical columns
df.drop(columns=['release_date','id','id_artists'],inplace = True)

# save genres 
# genres = genres_csv['genres']

# drop columns that arnt normalized (im lazy and it takes too long anyway)
# genres_csv.drop(columns=['mode','loudness','genres','duration_ms','tempo'],inplace=True)

In [3]:
# def calc_distance(row,genre):
#     distance = 0
#     for col in (genre.index):
#         distance += abs(row[col]-genre[col])
#     return distance

In [4]:
# This Took Way Too Long

# index = 0
# for i in range(len(df)):
#     min_diff = 1000
#     for j in range(len(genres_csv)):
#         diff = calc_distance(df.iloc[i],genres_csv.iloc[j])
#         if (diff <= min_diff):
#             index = j
#             min_diff = diff
#     print(i,genres[index],min_diff)
#     df['genre'][i] = genres[index]

In [5]:
# remove duplicates
print("Origional:", df.shape)
df = df.drop_duplicates()
print("Cleaned:", df.shape)

# label encode artists so It can select the same artist
enc = LabelEncoder()
df['artists'] = enc.fit_transform(df['artists'])

# normalize data
for i in df.columns[1:]:
    df[i] = df[i]/df[i].max()

Origional: (93135, 18)
Cleaned: (92767, 18)


In [6]:
X = df.drop(columns=['name']).values

In [7]:
neighbors = 20
nn = NearestNeighbors(n_neighbors=neighbors + 1)
nn.fit(X)

NearestNeighbors(n_neighbors=21)

In [8]:
# returns the nearest neighbor number of nearest songs given a song object
def song_suggester(song_obj):
    distance, neighbors = nn.kneighbors(np.array([song_obj]))
    suggestions = []
    for i in neighbors[0][1:]:
        suggestions.append([df['name'].iloc[i],df['artists'].iloc[i]*21583])
    sug = pd.DataFrame(suggestions,columns = ['song','artist'])
    sug['artist'] = enc.inverse_transform(sug['artist'].to_numpy().astype(int))
    return sug
# returns a song from song suggester from an exact name
def song(name):
    song = 'song'
    try:
        song = song_suggester(X[df['name']==name][0])
    except:
        print("Invalid Name Retard")
    return song
# Just call song(song_name) to get a dataframe with 20 suggestions [title,artist]

In [9]:
# example for creating similar songs
example = 29483

print('Song:',df['name'].iloc[example])

suggestions = song_suggester(X[example])

suggestions.head()

Song: Lillie


,song,artist
0,In The Evening (When The Sun Goes Down),['Mel Tormé']
1,Bailiff's Daughter of Islington,['Paul Clayton']
2,Luna Rossa,['Roberto Murolo']
3,Without A Love To Call My Own,['June Carter Cash']
4,White Christmas,['Merle Haggard']


In [10]:
df.head()

,name,popularity,duration_ms,explicit,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year
0,Carve,0.06,0.026437,0.0,0.934300,0.652834,0.4450,0.000000,-3.459922,1.0,0.464948,0.676707,0.7440,0.151151,0.127,0.430826,0.6,0.951014
1,Capítulo 2.16 - Banquero Anarquista,0.00,0.020458,0.0,0.295835,0.703441,0.2630,0.000000,-5.742153,1.0,0.986598,0.800201,0.0000,0.148148,0.655,0.419148,0.2,0.951014
2,Vivo para Quererte - Remasterizado,0.00,0.037841,0.0,0.403002,0.439271,0.1770,0.090909,-5.494163,1.0,0.052784,0.997992,0.0218,0.212212,0.457,0.535879,1.0,0.951014
3,El Prisionero - Remasterizado,0.00,0.036855,0.0,0.403002,0.324899,0.0946,0.636364,-7.253178,1.0,0.051959,0.998996,0.9180,0.104104,0.397,0.698437,0.6,0.951014
4,Lady of the Evening,0.00,0.033974,0.0,0.231895,0.406883,0.1580,0.272727,-4.383917,0.0,0.040206,0.992972,0.1300,0.311311,0.196,0.424124,0.8,0.951014


In [14]:
df.to_csv('data.csv',index=False)

In [16]:
joblib.dump(nn,'model.z')
joblib.dump(enc,'encoder.z')

['encoder.z']